# Universidad Autónoma de Yucatán

## Facultad de Matemáticas

### Natural Language Processing

**Teacher:** Dr. Jorge Carlos Reyes

**Student:** Dayan Bravo Fraga

# BERT

## Bidirectional Encoder Representations from Transformers

# Practice: "Spam Detection"

## Download Corpus from GitHub (only for Colab)

In [2]:
import sys

in_colab: bool = 'google.colab' in sys.modules
if in_colab:
    print('Is running in Colab')
    import os

    if not os.path.isfile('spam.csv'):
        import gdown

        print("Downloading Data")
        url = "https://raw.githubusercontent.com/dayan3847/natural_language_processing/dayan3847/task_final/spam.csv"
        gdown.download(url, quiet=False)
else:
    print('Is not running in Colab')

## Import Libraries

In [15]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

## Import Data

In [4]:
spam_data = pd.read_csv('spam.csv')
spam_data.head()

## Perform Basic Analysis

In [5]:
spam_data.groupby('target').describe()

# Change Target to (0, 1)

In [6]:
spam_data['target'] = spam_data['target'].apply(lambda x: 1 if x == 'spam' else 0)
spam_data.head()

## Train Test Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    spam_data['text'],
    spam_data['target'],
    random_state=0,
    # stratify=spam_data['target'],
)

## Creating Embedding using BERT

In [8]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [9]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [10]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [11]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [12]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [14]:
model.fit(x_train, y_train, epochs=10)

In [23]:
model.save('bert_model')

In [24]:
model = tf.keras.models.load_model('bert_model')

In [16]:
y_predicted = model.predict(x_test)
y_predicted = y_predicted.flatten()

In [18]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [20]:
len(y_predicted)

In [22]:
print('hello')
